# Creating Shapefiles for Web App - Custom_B

Inputs: 
1) Pagg_ReportingUnit.csv.  Contains reportinug unit info from WaDE database.

2) 

Paring HUC8 to ReportingUnitNativeID

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Set working directory
workingDir = "C:/Users/rjame/Documents/RShinyAppPractice/CreateAppShapefiles"
os.chdir(workingDir)

# Grab AggreagatedAmounts ReportingUnit.csv file.
reportingunits_input = pd.read_csv('SourceFiles/Pagg_ReportingUnit.csv')
df_1RU = pd.DataFrame(reportingunits_input)
df_1RU.head(3)

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,EPSGCodeCV
0,254,00-01-03,00-01-03,Curlew Valley,Subarea,UT,EPSG:4326
1,255,000-01-03,000-01-03,Clear Creek,Subarea,UT,EPSG:4326
2,256,00-07-02,00-07-02,Promontory Point,Subarea,UT,EPSG:4326


### California - Detailed Analysis Units by County

In [4]:
# Grab the CA Planning Area Shapefile.
# Paring RU_ID to ReportingUnitNativeID

CADAUCOshapefile_input = gpd.read_file('C:/Users/rjame/Documents/RShinyAppPractice/CreateAppShapefiles/SourceFiles/Custom/CA/WaDECADAU.shp')
dfs_CADAUCO = pd.DataFrame(CADAUCOshapefile_input)
dfs_CADAUCO.head(3)

,HR_Code,PA_Num,Shape_Leng,Shape_Le_1,Shape_Area,RU_ID,geometry
0,1,101,2.962973,2.962973,0.330267,DAU00125,"POLYGON ((-121.08710 41.99514, -120.70108 41.9..."
1,1,101,2.002975,2.002975,0.147869,DAU00147,"POLYGON ((-121.88226 42.00329, -121.44784 41.9..."
2,1,101,2.012166,2.012166,0.166514,DAU00247,"POLYGON ((-122.02221 42.00440, -121.94694 42.0..."


In [5]:
# Custom
# State: CA, Detailed Analysis Units by County
###########################################################################

# Create temporal dataframes for state specific and reportingunit type storage
df_1RU_Custom_CAdauco = df_1RU[(df_1RU.ReportingUnitTypeCV == 'Detailed Analysis Units by County') & (df_1RU.StateCV == 'CA')]

# retreive ReportingUnitUUID.
ReportingUnitUUIDdict = pd.Series(df_1RU_Custom_CAdauco.ReportingUnitUUID.values, index = df_1RU_Custom_CAdauco.ReportingUnitNativeID).to_dict()
def retrieveCountyName(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue
        try:
            outList = ReportingUnitUUIDdict[String1]
        except:
            outList = ''
    return outList
dfs_CADAUCO['ReportingUnitUUID'] = dfs_CADAUCO.apply(lambda row: retrieveCountyName(row['RU_ID']), axis=1)

# Merging temporal dataframes into one, using left-join.
dfs_CADAUCO = pd.merge(dfs_CADAUCO, df_1RU_Custom_CAdauco, left_on='ReportingUnitUUID', right_on='ReportingUnitUUID', how='left')

# Creating new output state specific dataframe with fields of interest.
dfs_2CADAUCO = pd.DataFrame() #empty dataframe
dfs_2CADAUCO['OBJECTID'] = dfs_CADAUCO.index
dfs_2CADAUCO['Shape'] = 'Polygon'
dfs_2CADAUCO['ReportingUnitID'] = dfs_CADAUCO['ReportingUnitID']
dfs_2CADAUCO['ReportingUnitUUID'] = dfs_CADAUCO['ReportingUnitUUID']
dfs_2CADAUCO['ReportingUnitNativeID'] = dfs_CADAUCO['ReportingUnitNativeID']
dfs_2CADAUCO['ReportingUnitName'] = dfs_CADAUCO['ReportingUnitName']
dfs_2CADAUCO['ReportingUnitTypeCV'] = dfs_CADAUCO['ReportingUnitTypeCV']
dfs_2CADAUCO['StateCV'] = dfs_CADAUCO['StateCV']
dfs_2CADAUCO['geometry'] = dfs_CADAUCO['geometry']

# view output
dfs_2CADAUCO.head(3)

,OBJECTID,Shape,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,geometry
0,0,Polygon,23643.0,CAag_RU67,DAU00125,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.08710 41.99514, -120.70108 41.9..."
1,1,Polygon,23644.0,CAag_RU68,DAU00147,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.88226 42.00329, -121.44784 41.9..."
2,2,Polygon,23645.0,CAag_RU69,DAU00247,Butte Valley,Detailed Analysis Units by County,CA,"POLYGON ((-122.02221 42.00440, -121.94694 42.0..."


In [6]:
# Merge dataframes
frames = [dfs_2CADAUCO]
outdf = pd.concat(frames)
outdf.head(3)

,OBJECTID,Shape,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,geometry
0,0,Polygon,23643.0,CAag_RU67,DAU00125,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.08710 41.99514, -120.70108 41.9..."
1,1,Polygon,23644.0,CAag_RU68,DAU00147,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.88226 42.00329, -121.44784 41.9..."
2,2,Polygon,23645.0,CAag_RU69,DAU00247,Butte Valley,Detailed Analysis Units by County,CA,"POLYGON ((-122.02221 42.00440, -121.94694 42.0..."


In [7]:
# drop NA rows
outdf = outdf.dropna(subset=['ReportingUnitID'])
outdf

,OBJECTID,Shape,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,geometry
0,0,Polygon,23643.0,CAag_RU67,DAU00125,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.08710 41.99514, -120.70108 41.9..."
1,1,Polygon,23644.0,CAag_RU68,DAU00147,Lost River,Detailed Analysis Units by County,CA,"POLYGON ((-121.88226 42.00329, -121.44784 41.9..."
2,2,Polygon,23645.0,CAag_RU69,DAU00247,Butte Valley,Detailed Analysis Units by County,CA,"POLYGON ((-122.02221 42.00440, -121.94694 42.0..."
3,3,Polygon,23646.0,CAag_RU70,DAU00347,Scott Valley,Detailed Analysis Units by County,CA,"POLYGON ((-122.81946 41.76864, -122.79461 41.7..."
4,4,Polygon,23647.0,CAag_RU71,DAU00447,Shasta Valley,Detailed Analysis Units by County,CA,"POLYGON ((-122.58104 41.82717, -122.56769 41.8..."
...,...,...,...,...,...,...,...,...,...
514,514,Polygon,24123.0,CAag_RU547,DAU40323,Upper Russian,Detailed Analysis Units by County,CA,"POLYGON ((-123.18216 39.24063, -123.18714 39.2..."
516,516,Polygon,24124.0,CAag_RU548,DAU40423,Middle Russian,Detailed Analysis Units by County,CA,"POLYGON ((-122.85634 38.86348, -122.84992 38.8..."
517,517,Polygon,24125.0,CAag_RU549,DAU40449,Middle Russian,Detailed Analysis Units by County,CA,"POLYGON ((-122.82249 38.85118, -122.81670 38.8..."
518,518,Polygon,24126.0,CAag_RU550,DAU40523,Dry Creek,Detailed Analysis Units by County,CA,"POLYGON ((-123.10947 38.87033, -123.11125 38.8..."


In [8]:
# Export the dataframe to a shapefile.
# Note: rememeber that shapefiles autofill in fields names, will need to fix field names in app upload.
dfsOut = gpd.GeoDataFrame(outdf, geometry='geometry') # covert to geodataframe
dfsOut.to_file("Processed_Shapefiles/P_Custom_B.shp") # export shape file